In [38]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN

np.random.seed(0)

config = configparser.ConfigParser()
config.read("/notebooks/notebooks/credentials.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [39]:
df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()
# df = pd.read_csv("/data/funds.csv")

In [4]:
# df.to_csv("/data/funds.csv", index=False)

In [40]:
df.head()

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,MEMBER_AGE,INVESTMENT_AGE_GROUP,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT
0,MID233585,Viola Nabours,Edith Cowan University,Male,Adelaide,South Australia,Y,FID000117,HIGH TRANSACTION FEE,-34.9275,...,36,Middle-aged,29,MORE_THAN_10_YEAR,2,1.0,1988-06-21,2001-03-26,2008-07-03,2053-06-21
1,MID234174,James Gillian,La Trobe University,Male,Sydney,New South Wales,N,FID000074,NO REASON IDENTIFIED,-33.8678,...,42,Middle-aged,23,MORE_THAN_10_YEAR,2,1.0,1982-09-30,2002-09-05,2008-07-09,2047-09-30
2,MID233455,Rima Jacobos,The University of Notre Dame Australia,Male,Esperance,Western Australia,N,FID000124,NO REASON IDENTIFIED,-33.8611,...,49,Middle-aged,16,MORE_THAN_10_YEAR,3,2.0,1975-06-10,2004-07-02,2009-12-12,2040-06-10
3,MID233456,Cammy Semidey,James Cook University,Male,Goonellabah,New South Wales,Y,FID000101,HIGH ACCOUNT FEE,-28.8167,...,43,Middle-aged,22,MORE_THAN_10_YEAR,2,1.0,1981-01-31,2002-06-08,2007-04-23,2046-01-31
4,MID233462,Elouise Kolmetz,University of Southern Queensland,Male,Albury,New South Wales,Y,FID000186,FUND REPUTATION DECLINING,-36.0806,...,39,Middle-aged,26,MORE_THAN_10_YEAR,2,1.0,1985-07-06,2002-09-12,2008-12-14,2050-07-06


In [41]:
df.isna().sum() / df.shape[0]

MEMBER_ID                                           0.000000
MEMBER_NAME                                         0.000000
MEMBER_EMPLOYMENT                                   0.000000
MEMBER_GENDER                                       0.000000
MEMBER_CITY_TOWN                                    0.000000
MEMBER_STATE                                        0.000000
MEMBER_CONTACT_VERIFIED                             0.000000
FUND_ID                                             0.000000
CHURN_REASON                                        0.000000
LATITUDE                                            0.000000
LONGITUDE                                           0.000000
COUNTRY                                             0.000000
COUNTRYCODE                                         0.000000
CAPITAL                                             0.000000
POPULATION                                          0.000000
FUND_TOTAL_ASSETS                                   0.000000
FUND_RETURN_TARGET_PERCE

NO_OF_CHURN column has almost 50% missing value, it wont be wise to impute these many values so will just drop it.                                     

In [42]:
frame = df[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()

In [43]:
frame["CHURN_FLAG"] = frame["CHURN_FLAG"].apply(lambda x: 1 if x =="Y" else 0)

In [8]:
# frame.groupby(["RETIREMENT_AGE_GROUP","CHURN_FLAG"])[["CHURN_FLAG"]].count()

The distribution of CHURN is pretty common bettween the employers and hence wont contribute inn modelling

In [9]:
# multiple = {i for i, j in dict(df["MEMBER_ID"].value_counts()).items() if j > 1 }
# multiple[multiple["value_counts"]>=2].index.to_list()

# MODEL TO PREDICT CHURN

In [44]:
numeric_features = ["FUND_TOTAL_ASSETS", 'FUND_RETURN_TARGET_PERCENTAGE','CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION','DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["MEMBER_GENDER", "MEMBER_STATE", "MEMBER_CONTACT_VERIFIED","INVESTMENT_RISK_CATEGORY",'FUND_RISK_LEVEL',
                        'FUND_RISK_CATEGORY',"NEG_NETRETURN_SINCE_INCEPTION", 'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', "TOTAL_FUNDS_INVESTED"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [45]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(n_estimators=500, max_depth=8, 
                                                                                 max_features=0.6,
                                                                                 bootstrap=True, max_samples=0.8))]
)



In [46]:
X = frame.drop("CHURN_FLAG", axis=1)
y = frame["CHURN_FLAG"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [47]:
clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.989


In [48]:
import pickle

# save
with open('churn_model.pkl','wb') as f:  ## use rb while reading the fie
    pickle.dump(clf,f)

In [49]:
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['FUND_TOTAL_ASSETS',
                                                   'FUND_RETURN_TARGET_PERCENTAGE',
                                                   'CASH_BENCHMARK_ALLOCATION',
                                                   'FIXED_INCOME_BENCHMARK_ALLOCATION',
                                                   'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION',
                                                   'INTERNATIONAL_LISTED_EQU...
                                                                                    score_func=<function chi2 at 0x7f96346fedc0>))]),
                                                  ['MEMBER_GENDER',
                                                   'MEMBER_STATE',
                                                   'MEMBER_CONTACT_VERIFIED',
                                                   'INVESTMENT_RISK_CATEGORY',
                                                   'FUND_RISK_LEVEL',
                                                   'FUND_RISK_CATEGORY',
                                                   'NEG_NETRETURN_SINCE_INCEPTION',
                                                   'INVESTMENT_AGE_GROUP',
                                                   'RETIREMENT_AGE_GROUP',
                                                   'TOTAL_FUNDS_INVESTED'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=8, max_features=0.6,
                                        max_samples=0.8, n_estimators=500))])

In [50]:
frame = clf.predict(frame)
df["CHURN_PREDICTED"] = frame

# MODEL_REASON

In [51]:
churned = df[df["CHURN_FLAG"] == "Y"]

In [52]:
churned.groupby(["CHURN_REASON"])[["CHURN_REASON"]].count() / churned.shape[0]*100

,CHURN_REASON
CHURN_REASON,
FUND REPUTATION DECLINING,29.070673
HIGH ACCOUNT FEE,16.143163
HIGH TRANSACTION FEE,21.015857
LIFE EVENT,5.547624
NO REASON IDENTIFIED,25.771735
POOR FUND PERFORMANCE,2.450949


In [53]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [54]:
# frame_id = churned[["FUND_ID",""]]
frame = churned[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED',"CHURN_REASON"]].copy()

In [55]:
churn_reason_dic = {reason:i for i, reason in enumerate(churned["CHURN_REASON"].unique())}
frame["CHURN_REASON"] = frame["CHURN_REASON"].apply(lambda x: churn_reason_dic[x])

In [56]:
numeric_features = ["FUND_TOTAL_ASSETS", 'FUND_RETURN_TARGET_PERCENTAGE','CASH_BENCHMARK_ALLOCATION',
       'FIXED_INCOME_BENCHMARK_ALLOCATION','DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
       'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION', 'INFRA_BENCHMARK_ALLOCATION',
       'COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS',
       'YEAR_5_RETURNS', 'YEAR_7_RETURNS', 'YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["MEMBER_GENDER", "MEMBER_STATE", "MEMBER_CONTACT_VERIFIED","INVESTMENT_RISK_CATEGORY",'FUND_RISK_LEVEL',
                        'FUND_RISK_CATEGORY',"NEG_NETRETURN_SINCE_INCEPTION", 'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', "TOTAL_FUNDS_INVESTED"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [57]:
multi_lable_clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=500, max_depth=8, max_features=0.6, bootstrap=True,
                                                             max_samples=0.8))
clf2 = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", multi_lable_clf)]
)


X = frame.drop("CHURN_REASON", axis=1)
y = frame["CHURN_REASON"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape)

clf2.fit(X_train, y_train)
print("model score: %.3f" % clf2.score(X_test, y_test))

(73963, 29)
model score: 1.000


In [58]:
frame_pred = clf2.predict(frame)
churned["CHURN_REASON_PREDICTED"] = frame_pred

In [62]:
churned.drop_duplicates()

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT,CHURN_PREDICTED,CHURN_REASON_PREDICTED
0,MID233585,Viola Nabours,Edith Cowan University,Male,Adelaide,South Australia,Y,FID000117,HIGH TRANSACTION FEE,-34.9275,...,29,MORE_THAN_10_YEAR,2,1.0,1988-06-21,2001-03-26,2008-07-03,2053-06-21,1,0
1,MID234174,James Gillian,La Trobe University,Male,Sydney,New South Wales,N,FID000074,NO REASON IDENTIFIED,-33.8678,...,23,MORE_THAN_10_YEAR,2,1.0,1982-09-30,2002-09-05,2008-07-09,2047-09-30,1,1
2,MID233455,Rima Jacobos,The University of Notre Dame Australia,Male,Esperance,Western Australia,N,FID000124,NO REASON IDENTIFIED,-33.8611,...,16,MORE_THAN_10_YEAR,3,2.0,1975-06-10,2004-07-02,2009-12-12,2040-06-10,1,1
3,MID233456,Cammy Semidey,James Cook University,Male,Goonellabah,New South Wales,Y,FID000101,HIGH ACCOUNT FEE,-28.8167,...,22,MORE_THAN_10_YEAR,2,1.0,1981-01-31,2002-06-08,2007-04-23,2046-01-31,1,2
4,MID233462,Elouise Kolmetz,University of Southern Queensland,Male,Albury,New South Wales,Y,FID000186,FUND REPUTATION DECLINING,-36.0806,...,26,MORE_THAN_10_YEAR,2,1.0,1985-07-06,2002-09-12,2008-12-14,2050-07-06,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335994,MID116558,Raul Cryer,The University of Notre Dame Australia,Male,Darwin,Northern Territory,N,FID000114,HIGH TRANSACTION FEE,-12.4381,...,24,MORE_THAN_10_YEAR,2,1.0,1983-12-20,2003-11-10,2007-09-11,2048-12-20,1,0
335995,MID116606,Lonna Perin,University of Canberra,Female,Hobart,Tasmania,Y,FID000126,HIGH TRANSACTION FEE,-42.8806,...,24,MORE_THAN_10_YEAR,2,1.0,1983-10-23,2000-02-15,2008-06-30,2048-10-23,1,0
335996,MID116699,Tiffiny Flister,Queensland University of Technology,Male,Perth,Tasmania,Y,FID000005,HIGH TRANSACTION FEE,-31.9559,...,6,IN_NEXT_5_YEAR,2,1.0,1965-06-24,2001-10-24,2009-01-11,2030-06-24,1,0
335997,MID116750,Adell Lueckenbach,RMIT University,Male,Hobart,Tasmania,N,FID000166,NO REASON IDENTIFIED,-42.8806,...,38,MORE_THAN_10_YEAR,2,1.0,1997-06-06,2002-08-29,2009-03-18,2062-06-06,1,1


In [59]:
import pickle

# save
with open('reason_model.pkl','wb') as f:  ## use rb while reading the fie
    pickle.dump(clf2,f)

# FUND RECOMMENDATION

In [24]:
fund_allocation = session.table("FUND_ALLOCATION_MASTER").to_pandas()
fund_master = session.table("FUND_MASTER").to_pandas()
fund_performance = session.table("FUND_PERFORMANCE_MASTER").to_pandas()

In [25]:
fund = pd.merge(fund_allocation, fund_master, on="FUND_ID")
fund_total = pd.merge(fund, fund_performance, on="FUND_ID")
fund_charge = [float(i) for i in fund_total["FUND_FEES_CHARGES"] if i != 'Not Available']
avg_fund_charge = sum(fund_charge) / len(fund_charge)

fund_total["FUND_FEES_CHARGES"] = fund_total["FUND_FEES_CHARGES"].apply(lambda x: avg_fund_charge if x == 'Not Available' else x)
fund_total["FUND_FEES_CHARGES"] = fund_total["FUND_FEES_CHARGES"].astype(float) 

fund_meta = fund_total[["FUND_ID","FUNDNAME"]]
fund_cluster = fund_total.drop(["FUND_ID","FUNDNAME","FUND_NAME_y","FUND_TRUSTEE","FUND_INCEPTION_DATE"], axis=1)

In [26]:
fund_data = pd.get_dummies(fund_cluster)

fund_data = fund_data.replace(False, 0)
fund_data = fund_data.replace(True, 1)

In [27]:
clustering = DBSCAN(eps=2, min_samples=3).fit(fund_data)

In [28]:
fund_total["cluster"] = clustering.labels_
fund_total["avg_return"] = fund_total[["YEAR_1_RETURNS","YEAR_3_RETURNS",
                                 "YEAR_5_RETURNS","YEAR_7_RETURNS",
                                 "YEAR_10_RETURNS"]].apply(lambda x:
                                                        (x[0] + (0.9*x[1]) + (0.75*x[2]) + (0.6*x[3]) + (0.55*x[4]))/5, axis=1)

In [29]:
def recommendation(fund_id, reason):
    if reason == 'NO REASON IDENTIFIED' or reason == "NOT APPLICABLE":
        return "Not Applicable"
    elif reason == 'HIGH ACCOUNT FEE' or reason == 'HIGH TRANSACTION FEE':
        clus_frame = fund_total[fund_total["FUND_ID"] == fund_id]
        clus = list(clus_frame["cluster"])[0]
        clus_frame["charge"] = clus_frame[["SUPER_FEES","PENSION_FEES","FUND_FEES_CHARGES"]].apply(lambda x: (x[0]+x[1]+x[2])/3, axis=1)
        total_charge = list(clus_frame["charge"])[0]
        group = fund_total[fund_total["cluster"] == clus]
        group["avg_fee"] = group[["SUPER_FEES","PENSION_FEES","FUND_FEES_CHARGES"]].apply(lambda x: (x[0]+x[1]+x[2])/3, axis=1)
        group["rec"] = group["avg_fee"].apply(lambda x: 1 if x < total_charge else 0)
        group = group.sort_values("avg_return")
        recommendations = list(group[group["rec"] == 1]["FUND_ID"])
        if fund_id in recommendations:
            recommendations.remove(fund_id)
        recommendations = recommendations[:5]
        if len(recomendations) > 1:
            recomendations = np.random.choice(recomendations,1)[0]
        else:
            return "Not Applicable"
        return recommendations

    elif reason == 'FUND REPUTATION DECLINING' or reason == 'POOR FUND PERFORMANCE':
        clus_frame = fund_total[fund_total["FUND_ID"] == fund_id]
        clus = list(clus_frame["cluster"])[0]
        group = fund_total[fund_total["cluster"] == clus]
        group = group.sort_values("avg_return")
        recommendations = list(group["FUND_ID"])
        if fund_id in recommendations:
            recommendations.remove(fund_id)
        recommendations = recommendations[:5]
        if len(recomendations) > 1:
            recomendations = np.random.choice(recomendations,1)[
        else:
            return "Not Applicable"
        return recommendations
    
    else:
        return "Not Applicable"
        
            
            
            
rec = recommendation("FID000014","HIGH ACCOUNT FEE")
print(len(rec))

1


In [30]:
from tqdm import tqdm

In [31]:
rec_fund = []
for fund_id, f_reason, churn in tqdm(zip(df["FUND_ID"], df["CHURN_REASON"], df["CHURN_PREDICTED"])):
    if churn == 1:
        rec = recommendation(fund_id,f_reason)
        rec_fund.append(rec)
    else:
        rec_fund.append([])
    

335999it [06:17, 890.81it/s] 


In [32]:
df["funds_rec"] = rec_fund

In [33]:
df.to_csv("/data/funds_recommendated.csv", index=False)

In [34]:
fund_total.to_csv("/data/FUNDS_COMPLETE_DATA.csv", index=False)

In [47]:
# for i in df["funds_rec"]:
#     if i != []:
#         print(i)

In [36]:
df[df["CHURN_PREDICTED"] == 1]

,MEMBER_ID,MEMBER_NAME,MEMBER_EMPLOYMENT,MEMBER_GENDER,MEMBER_CITY_TOWN,MEMBER_STATE,MEMBER_CONTACT_VERIFIED,FUND_ID,CHURN_REASON,LATITUDE,...,RETIREMENT_AGE,RETIREMENT_AGE_GROUP,TOTAL_FUNDS_INVESTED,NO_OF_CHURN,MEMBER_DOB_DT,ALLOCATION_DT,CHURN_DT,RETIREMENT_DT,CHURN_PREDICTED,funds_rec
0,MID233585,Viola Nabours,Edith Cowan University,Male,Adelaide,South Australia,Y,FID000117,HIGH TRANSACTION FEE,-34.9275,...,29,MORE_THAN_10_YEAR,2,1.0,1988-06-21,2001-03-26,2008-07-03,2053-06-21,1,[]
1,MID234174,James Gillian,La Trobe University,Male,Sydney,New South Wales,N,FID000074,NO REASON IDENTIFIED,-33.8678,...,23,MORE_THAN_10_YEAR,2,1.0,1982-09-30,2002-09-05,2008-07-09,2047-09-30,1,[]
2,MID233455,Rima Jacobos,The University of Notre Dame Australia,Male,Esperance,Western Australia,N,FID000124,NO REASON IDENTIFIED,-33.8611,...,16,MORE_THAN_10_YEAR,3,2.0,1975-06-10,2004-07-02,2009-12-12,2040-06-10,1,[]
3,MID233456,Cammy Semidey,James Cook University,Male,Goonellabah,New South Wales,Y,FID000101,HIGH ACCOUNT FEE,-28.8167,...,22,MORE_THAN_10_YEAR,2,1.0,1981-01-31,2002-06-08,2007-04-23,2046-01-31,1,"[FID000102, FID000100]"
4,MID233462,Elouise Kolmetz,University of Southern Queensland,Male,Albury,New South Wales,Y,FID000186,FUND REPUTATION DECLINING,-36.0806,...,26,MORE_THAN_10_YEAR,2,1.0,1985-07-06,2002-09-12,2008-12-14,2050-07-06,1,"[FID000103, FID000034, FID000136, FID000135, F..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335994,MID116558,Raul Cryer,The University of Notre Dame Australia,Male,Darwin,Northern Territory,N,FID000114,HIGH TRANSACTION FEE,-12.4381,...,24,MORE_THAN_10_YEAR,2,1.0,1983-12-20,2003-11-10,2007-09-11,2048-12-20,1,"[FID000113, FID000115, FID000116]"
335995,MID116606,Lonna Perin,University of Canberra,Female,Hobart,Tasmania,Y,FID000126,HIGH TRANSACTION FEE,-42.8806,...,24,MORE_THAN_10_YEAR,2,1.0,1983-10-23,2000-02-15,2008-06-30,2048-10-23,1,"[FID000103, FID000097, FID000096, FID000146, F..."
335996,MID116699,Tiffiny Flister,Queensland University of Technology,Male,Perth,Tasmania,Y,FID000005,HIGH TRANSACTION FEE,-31.9559,...,6,IN_NEXT_5_YEAR,2,1.0,1965-06-24,2001-10-24,2009-01-11,2030-06-24,1,[]
335997,MID116750,Adell Lueckenbach,RMIT University,Male,Hobart,Tasmania,N,FID000166,NO REASON IDENTIFIED,-42.8806,...,38,MORE_THAN_10_YEAR,2,1.0,1997-06-06,2002-08-29,2009-03-18,2062-06-06,1,[]
